<a href="https://colab.research.google.com/github/suuuuwimmer/study/blob/master/5-3.%20RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 트리의 앙상블

## 랜덤포레스트

###정형 데이터와 비정형 데이터

**정형 데이터**는 효율적인 구조로 이루어져 있어 바로 사용할 수 있는 데이터를 말한다.\
**비정형 데이터**는 텍스트, 사진, 음악 등 데이터 베이스나 엑셀로 표현하기 어려운 데이터를 말한다.

정형 데이터에는 **앙상블 학습 알고리즘**이 가장 효율적이다. 이 알고리즘은 대부분 결정 트리로 이루어져있다.\
앙상블 학습 알고리즘 중 랜덤 포레스트는 안정적인 성능으로 널리 사용되고 있다.\
랜덤 포레스트는 결정 트리를 랜덤하게 만들어 숲을 만든다. 그리고 각 결정 트리의 예측을 사용해 최종 예측을 만든다.\
이때 **부트스트랩 샘플**이 만들어지는데 데이터 세트에서 중복을 허용해 샘플링하는 방식을 말한다.

###랜덤 포레스트 구현

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

데이터를 가져오고 훈련 세트와 테스트 세트로 분활함

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


cross_validate() 함수를 사용해 교차 검증을 수행\
출력된 결과를 보아 과소적합됨을 확인할 수 있음

In [ ]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


랜덤 포레스트는 결정 트리의 앙상블이기 때문에 여러 중요한 매개변수를 제공한다.\
criterion, max_depth, max_feature, min_samples_split, min_impurity_decrease, min_samples_leaf 등이 있다.

In [ ]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


랜덤 포레스트는 자체적으로 모델을 평가하는 기능이 있다.\
부트스트랩 샘플에 포함되지 않고 남는 OOB 샘플이 있는데 이를 이용해 결정 트리를 평가하는 것이다.\
이를 이용하기 위해서는 oob_score 매개변수를 활성화 시키고 oob의 점수를 출력하면 된다.

## 엑스트라트리

엑스트라 트리는 랜덤 포레스트와 매우 유사하게 동작한다.\
그러나 둘의 차이점은 엑스트라 트리가 부트스트랩 샘플을 사용하지 않는다는 점이다. 즉, 결정 트리를 만들때 전체 훈련 세트를 사용한다.\
splitter='random'인 결정트리가 바로 엑스트라 트리이다.

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [ ]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


## 그레이디언트 부스팅

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [ ]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [ ]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15887763 0.6799705  0.16115187]


## 히스토그램 기반 부스팅

In [ ]:
# 사이킷런 1.0 버전 아래에서는 다음 라인의 주석을 해제하고 실행하세요.
# from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [ ]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [ ]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [ ]:
hgb.score(test_input, test_target)

0.8723076923076923

#### XGBoost

In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9558403027491312 0.8782000074035686


#### LightGBM

In [ ]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


0.935828414851749 0.8801251203079884
